# Using an LLM Agent with Web Search to Classify Sports Teams from a CSV File

This notebook demonstrates how to use a LangChain-based LLM agent with a web search tool to classify sports teams listed in a CSV file. Each row in the CSV contains a team name. The agent will use real-time web search to determine the sport associated with each team and add a corresponding label.

## Step 1: Install Required Packages

We will use the LangChain framework, OpenAI for LLM access, and the Tavily web search integration. Pandas will be used to work with CSV files.

In [ ]:
!pip install --quiet langchain_openai pandas tavily-python langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.4/63.4 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.4/438.4 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 1.7 MB/s eta 0:00:00


## Step 2: Set Up API Keys

You will need:
- An OpenAI API key
- A Tavily API key (sign up at https://app.tavily.com/)

These are required to access the LLM and the web search service.

In [ ]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")
os.environ["TAVILY_API_KEY"] = getpass.getpass("Enter your Tavily API key: ")

Enter your OpenAI API key: ··········
Enter your Tavily API key: ··········


## Step 3: Initialize the Agent with Web Search Tool

We will set up an LLM agent with the Tavily web search tool. The agent will decide when to invoke the search tool to classify each team.

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.agents import initialize_agent, AgentType
from langchain_community.tools.tavily_search import TavilySearchResults

# Initialize model and tools
llm = ChatOpenAI(temperature=0)
search_tool = TavilySearchResults()
agent = initialize_agent([search_tool], llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

<ipython-input-3-c4d191d3ca0e>:8: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent([search_tool], llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)


## Step 4: Load the CSV File

We load the CSV file that contains the names of sports teams. Each row has one team name under a column named 'Team'.

In [ ]:
import pandas as pd

# Path to your CSV file
csv_path = "sports_teams.csv"
df = pd.read_csv(csv_path)
df.head()

,Team,Country,City,Stadium Capacity
0,Los Angeles Lakers,USA,Los Angeles,19000
1,Manchester United,UK,Manchester,74000
2,Toronto Maple Leafs,Canada,Toronto,18600
3,Golden State Warriors,USA,San Francisco,19500
4,New England Patriots,USA,Boston,65878


## Step 5: Classify Each Team Using the Agent

The agent will search the web and respond with the sport that each team plays. We apply this to each team in the dataset.

In [ ]:
# Define a function that sends a query to an LLM agent to classify the sport associated with a given team name.
# The agent uses web search and reasoning to identify the sport, and returns only the name of the most well-known one.
# If an error occurs during the query process, it returns an error message instead.
def classify_team_with_agent(team_name):
    try:
        query = f"""What sport does the team '{team_name}' play? only return
        the sport name, if there are multiple sports associated with the name,
        only return the most famous one"""
        return agent.run(query)
    except Exception as e:
        return f"error: {str(e)}"

# Apply the classification function to the first 3 rows of the 'Team' column in the DataFrame.
# The result is stored in a new column called 'Sport'. This is useful for testing before applying to the full dataset.
df["Sport"] = df[:3]["Team"].apply(classify_team_with_agent)

df.head()

<ipython-input-7-fb2c45b2fd22>:6: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  return agent.run(query)




> Entering new AgentExecutor chain...
I should use the search engine to find the sport associated with the team 'Los Angeles Lakers'.
Action: tavily_search_results_json
Action Input: Los Angeles Lakers sport
Observation: [{'title': 'Los Angeles Lakers - Wikipedia', 'url': 'https://en.wikipedia.org/wiki/Los_Angeles_Lakers', 'content': "The Los Angeles Lakers are an American professional basketball team based in Los Angeles. The Lakers compete in the National Basketball Association (NBA) as a member of the Pacific Division of the Western Conference. The Lakers play their home games at Crypto.com Arena, an arena they share with the Los Angeles Sparks of the Women's National Basketball Association (WNBA) and the Los Angeles Kings of the National Hockey League (NHL).[10] The Lakers are one of the most successful teams in the [...] Los Angeles Memorial Sports Arena. [...] entertainment as well as sport.[16] Second, the Lakers drafted Magic Johnson first overall in the 1979 NBA draft. The c

,Team,Country,City,Stadium Capacity,Sport
0,Los Angeles Lakers,USA,Los Angeles,19000,Basketball
1,Manchester United,UK,Manchester,74000,NaN
2,Toronto Maple Leafs,Canada,Toronto,18600,NaN
3,Golden State Warriors,USA,San Francisco,19500,NaN
4,New England Patriots,USA,Boston,65878,NaN


## Step 6: Save the Updated Table

The resulting DataFrame will be saved to a new CSV file with the added sport classification column.

In [ ]:
output_path = "classified_teams_with_web_search.csv"
df.to_csv(output_path, index=False)
print(f"Saved results to {output_path}")

Saved results to classified_teams_with_web_search.csv
